# 딥러닝 (3) 사전과제
## 인트로<br>
#### <span style='background-color: #fff5b1'> 1. RNN의 한계와 어텐션의 등장 </span>

**RNN 복습**<br>
- 앞뒤 내용이 서로 연관된 데이터를 기억해, 이전 단계에서 처리한 정보를 현재 입력에 활용      
➡️ 기울기 소실 문제   

**LSTM 복습**<br>
- 이전 단계에서 새로운 중요한 정보는 input gate를 통해 cell state로 입력하고, 중요하지 않으면 forget gate를 통해 잊음   
➡️ 기울기 소실 문제 (초반부 데이터들이 forget gate 통과 횟수 더 많음)    

**어텐션 등장**<br>
- seq2seq 병목 현상 해결을 위해 등장   
➡️ <u>병렬화</u> : 출력 시점마다 새로운 context vector를 동적으로 계산   

**어텐션 과정**<br>
1. 입력을 벡터화   
2. 각 요소별 중요도 점수 산출   
    - 관련도 채점 (score) : 단어 간 관계 / 유사성 측정    
    - 중요도 배분 (소프트 맥스) : score를 0과 1 사이 값으로 변환해 가중치를 만듦    

3. 중요도에 따른 입력 정보 조합 : 어텐션 가중치에 따라 정보에 집중 / 무시   

**어텐션 장점**<br>
(+)     
- 모든 단어에서 다른 모든 단어로 연결고리를 만들어, 멀리 떨어져 있어도 정보 손실 없이 파악 가능 ➡️ 시간 경과에 대한 유연성    
- 픽셀 간 거리와 무관하게 한 번에 전역적인 관계 학습 가능 ➡️ 공간 유연성 
- 문장 내 모든 단어의 관계를 한 번에 독립적으로 계산 가능 ➡️ 병렬화

<br>

#### <span style='background-color: #fff5b1'> 2. Attention is All You Need </span>
- 배경: 인코더~디코더까지 어텐션만을 사용하는 모델을 만든다면?    
- 주요 내용: 병렬화 가능    
- 파급 효과: LLM 탄생 & NLP의 표준으로 자리 잡음 & 타 분야로 확장   

<br>

**트랜스포머**<br>
: RNN의 순차적 계산 방식과 달리, 어텐션만으로 문장의 의미와 구조를 파악하는 모델    
- Self-Attention : 문장 내 단어 간 관계의 중요도 파악   
- Multi-Head Attention : 셀프 어텐션 여러 개를 동시에, 서로 다른 관점에서 실행    
- Positional Encoding : 단어의 위치 정보 포함   
➡️ 인코더, 디코더까지 RNN이 아닌 어텐션으로!  

## Self-Attention<br>
#### <span style='background-color: #fff5b1'> 1. Query, Key, Value </span>
**query**<br>
: 질문 / 요청 = 알고 싶거나, 초점을 맞추는 대상    

**key**<br>
: 검색 대상이 되는 모든 정보가 달고 있는 이름표 또는 인덱스    

**value**<br>
: 키와 한 쌍으로 묶인 실제 내용   

<br>

#### <span style='background-color: #fff5b1'> 2. Cross Attention vs. Self-Attention </span>

Self-Attention = 문장 스스로가 문맥을 이해하는 과정    
<br>

구분 | Cross Attention | Self-Attention    
----- | ----- | -----     
핵심 아이디어 | 번역할 문장 참고하며 단어 생성 | 문장 자체로 내부 관계 파악    
작동 방식 | 한 시퀀스가 다른 시퀀스를 참고해 정보 생성 | 문장 내 모든 단어 간 관계 계산     
. | 디코더가 인코더 참고 |   
참조 대상 | 서로 다른 시퀀스 | 동일 시퀀스    
기반 아키텍처 | RNN | 트랜스포머     
병렬 처리 | 불가능 | 가능     

#### <span style='background-color: #fff5b1'> 3. Scaled Dot-Product Attention </span>

**Dot-Product Attention**<br>
- Attention score 계산 방법 중 하나로, 쿼리와 키 벡터를 내적해 유사도 산출  
➡️ 계산된 유사도에 소프트 맥스 함수를 적용해 어텐션 가중치 산출 후, 최종 결과값 계산     
=> 벡터 차원이 클수록 내적 결과가 너무 크거나 작아지는 문제 존재 [기울기 소실 문제]  
<br>

**Scaled Dot-Product Attention**<br>
- 기존 dot-product attention 과정에서 내적 진행 후 스케일링 과정 추가하여 어텐션 가중치 산출하는 방식       

## Multi-Head Attention<br>

- 인코딩된 토큰을 아키텍처가 이해할 때 사용됨   
<br>

#### <span style='background-color: #fff5b1'> 1. Why? </span>

- 단어 간 관계를 파악하기 위해 하나의 가중치 행렬을 학습하고 사용하는 Single-Head Attention과 달리, 여러 차원으로 나누어 병렬 학습    

#### <span style='background-color: #fff5b1'> 2. 작동 방식 </span>

- 단어의 차원 / 어텐션 헤드의 수 / 각 헤드의 차원 = 512 / 8 / 64로 가정    
<br>

1. **분할**     
- 512차원의 벡터를 8개의 서로 다른 관점으로 분할   
➡️ 8개의 독립적인 64차원 벡터로 분할     

2. **병렬 어텐션 계산 (각 헤드가 분석 수행)**     
- 8개의 헤드가 병렬적으로 Scaled dot product attention 계산    
➡️ 헤드 1의 관점에서 문맥을 이해해 64차원의 결과 벡터 도출   

3. **결합 및 최종 투영 (전문가 분석 결과 취합)**     
- 8개의 헤드가 각각 도출한 결과를 하나로 결합하여 최종 투영    

## Transformer 전체 아키텍처<br>
#### <span style='background-color: #fff5b1'> 1. 트랜스포머의 전처리 단게 </span>

<u>**텍스트 입력 ➡️ 텍스트 토큰화 ➡️ 토큰 임베딩 ➡️순서 정보 추가 (Positional Encoding)**</u> <br>

**토큰화**    
= 입력한 텍스트를 모델이 처리하는 단위인 토큰으로 나눔    

**임베딩**     
= 토큰화된 각 단위를 숫자 벡터로 변환 [단어 의미를 공간적으로 표현]    
- 단어에 대한 기본적 속성 제공   

**positional encoding**    
= 순서 정보를 임베딩 벡터에 추가    
- 각 단어의 임베딩 좌표에 일관된 순서를 따르는 다른 숫자를 추가해 순서 정보 기록

#### <span style='background-color: #fff5b1'> 2. 인코더와 디코더 </span>
**인코더**<br>
- 입력 문장을 이해하고 요약된 의미 벡터로 변환   
- 전처리 과정 (토큰화, 임베딩, positional encoding) + 인코더 레이어(multi-head attention, feed forward layer)   

**인코더 레이어 구성**<br>
-<u>Feed-Forward Layer</u> : 입력 벡터의 차원을확장하고 비선형 변환을 적용해 새로운 표현 생성      
- 잔차 연결 : 입력 값을 그대로 출력에 더함 ➡️ 기울기 소실 및 폭주 방지   
- 층 정규화 : 입력 벡터의 평균과 분산으로 정규화 ➡️ 학습 안정화 & 모델의 빠른 수렴    

<br>

**디코더**<br>
- 인코더가 분석한 입력 문장의 의미 벡터를 받아 출력 문장을 순차적으로 생성    

**디코더 레이어 구성**<br>

(1) Masked Multi-Head Attention (Cross-Attention)     
= 미래 시점의 단어 정보 참고하지 못하도록 마스크 적용    
= 현재 단어보다 오른쪽에 있는 토큰의 어텐션 값을 0으로 만듦  

(2) Encoder-Decoder Multi-Head Attention (Cross-Attention)     
= 디코더가 인코더의 출력을 참고하며 현재 생성 중이 단어를 입력 문장의 의미와 연결     
- 입력 문장 중 디코더가 주목해야 할 부분을 학습   

(3) Feed-Forward Layer        

<br>

**인코더와 디코더 비교**<br>
내용 | 인코더 | 디코더   
------ | ------ | ------    
정보 흐름 | 양방향 (입력 시퀀스 전체를 한 번에) | 단방향 (특정 시점의 예측은 과거 토큰에 의존)     
병렬 처리 | 병렬 처리 가능 | 학습 시 병렬 but <u>출력 생성 시 순차적</u>     
입출력 차원 | 동일 | 동일     

#### <span style='background-color: #fff5b1'> 3. 전체 모델 및 데이터 흐름 </span>
1. 입력 준비 : 문장을 [의미 + 위치] 벡터로 변환   
    - 임베딩 & 포지셔널 인코딩    

2. 인코더 : 입력 문장의 문맥적 의미 이해   
    - 셀프 어텐션    

3. 디코더 : 번역 문장을 한 단어씩 생성